# Titanic Survival Prediction


Dzisiaj przyjrzymy się, jak wygląda proces tworzenia modelu. Nie jest to jedyna właściwa scieżka i ten notatnik jest jedynie wstępem do poważniejszych analiz.
Pod lupę weźmiemy wyzwanie [Titanic survival prediction challenge](http://www.kaggle.com/c/titanic-gettingStarted) z portalu [Kaggle](http://www.kaggle.com).
Nasza analiza prowadzona będzie z góry na dół, zaczniemy od głębszego zajrzenia do danych, następnie zbudujemy model i zastanowimy się, jak go poprawić.

[Źródło, z którego korzystałem](https://github.com/wehrley/wehrley.github.io/blob/master/SOUPTONUTS.md)
[Inne źródło](https://www.kaggle.com/c/titanic/details/getting-started-with-python)

### Narzędzia potrzebne do uczestniczenia w zajęciach i ich instalacja

Polecam działanie w środowisku wirtualnym (virtualenv, conda, itp.). Standardowa instalacja pakietów wygląda tak:

```shell
Instalujemy condę, odpalamy i gotowe
```

lub

```shell
Tworzymy wirtualne środowisko, a potem
pip2 install ipython notebook matplotlib pandas statsmodels

```

### O notebooku

Włączamy notebooka i wybieramy dostarczony przeze mnie plik. Pora na małą prezentację możliwości tego oprogramowania.
Wiele z nich pochodzi z IPythona. Każdą komórkę możemy odpalić za pomocą kombinacji Shift + Enter

In [ ]:
# Wbudowane wsparcie dla popularnych pakietów, m.in. matplotlib, 

%matplotlib inline
import numpy as np
from matplotlib.pyplot import *
x = np.linspace(0, 3*np.pi, 500)
plot(x, np.sin(x**2))
_ = title("Taki tam wykresik")

In [ ]:
# Wbudowane wsparcie dla dokumentacji
np.linspace?

In [ ]:
 # Wbudowane podstawowe komendy shellowe

In [ ]:
ls

In [ ]:
# Wbudowany debugger (ipdb).
%debug 

In [ ]:
%pdb 1
1/0

In [ ]:
# Wbudowany profiler
def all_fibbos(n):
    return [(fibbo(x),fibbo2(x)) for x in xrange(1,n)]

def fibbo2(n):
    a,b = 1,1
    for i in range(n-1):
       a,b = b,a+b
    return a

def fibbo(n):
    if n==1 or n==2:
        return 1
    return fibbo(n-1)+fibbo(n-2)

%prun all_fibbos(25)
 


#Wsparcie dla Markdown:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |  1600 |
| col 2 is      | centered      |    12 |

# Wprowadzenie

W wyzwaniu _Titanic Survival Prediction_ naszym zadaniem jest przewidzenie, którzy z pasażerów Titanica przeżyją, a którzy nie. [Wiki o katastrofie](https://pl.wikipedia.org/wiki/RMS_Titanic#Katastrofa)

Na poczatku postarajmy się sami zgadnąć, co wpływało na to, że jedni przeżyli, a drudzy nie. Jak myślicie, jakie były kluczowe czynniki?



In [ ]:
# Sugestie

#
#
#
#

# Wczytujemy dane i je wizualizujemy

Jedną z kluczowych rzeczy, które wpływają na skuteczność i szybkość wyprodukowania modeli jest porządne przygotowanie zbioru danych do dalszego przetworzenia i ich zrozumienie. Na tym etapie często można zauważyć nieoczywiste zależności, które przydadzą nam się przy budowie modeli. Jeżeli wydaje wam się, że przeznaczyliście wystarczająco czasu na ten etap, to znaczy, że należy przeznaczyć jeszcze drugie tyle.

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

import csv
import numpy as np
import pandas as pd

df = pd.read_csv('train.csv')

In [ ]:
df

In [ ]:
%load missmap.py

In [ ]:
_ = missmap(df)

Widzimy, że dwie cechy mają braki w danych. Brakami w wartościach dotyczących wieku nie musimy się przejmować - są one dość małe, ale niedostępne dane o numerze kabiny mogą nam przeszkadzać dalej. Nie oznacza to jednak, że nie będziemy mogli używać tej cechy w dalszej analizie!

Przeanalizujmy dystrybucję poszczególnych cech:

In [ ]:
ax = df['Survived'].plot.hist(bins=2,title='Ocaleni')
ax.set_xticks([0.25,0.75])
ax.set_xticklabels(['Zgineli', 'Przezyli'])
_ = ax.set_ylabel('Licznosc')

In [ ]:
ax = df['Pclass'].plot.hist(bins=3,title='Klasa')
ax.set_xticks([1.33,2,2.67])
ax.set_xticklabels(['Pierwsza', 'Druga', 'Trzecia'])
_ = ax.set_ylabel('Licznosc')

In [ ]:
df['Sex'].value_counts()


In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

def labelizer(x):
    klasa = {'3' : 'trzeciej', '2': 'drugiej', '1': 'pierwszej'}
    los = {'0': 'zginęli', '1': 'przeżyli'}
    return 'Z klasy ' + klasa[x[0]] + ', ' + los[x[1]]

ax = mosaic(df, ['Pclass', 'Survived'], labelizer=labelizer,
            title='Zależność pomiędzy wyborem klasy, a szansami przeżycia',
            axes_label=False)
ax[0].set_size_inches((10,5))

Zależność widoczną na powyższym wykresie da się wytłumaczyć. Spójrzmy na rozkład koji:

![](cutaway.gif)

Oraz na linię wody podczas tonięcia:

![](Titanic_side_d_deck.png)

Tytanik zaczął tonąć 20 minut przed północą i możemy przyjąć, że większość pasażerów znajdowała się wtedy w swoich kabinach

In [ ]:
def labelizer(x):
    plec = {'male' : 'Mężczyźni', 'female': 'Kobiety'}
    los = {'0': 'zginęli/ły', '1': 'przeżyli/ły'}
    return  plec[x[0]] + ', ' + los[x[1]]

ax = mosaic(df, ['Sex', 'Survived'], labelizer=labelizer,
            title='Zależność pomiędzy wyborem klasy, a szansami przeżycia',
            axes_label=False)
ax[0].set_size_inches((10,5))

Zgodnie z regułami i etykietą pierwszeństwo we wstępie na łodzie ratunkowe miały kobiety i dzieci.

In [ ]:
ax = df.boxplot(['Age'], by='Survived')
ax.set_title('Zależność wieku od szans na przeżycie')
ax.set_xlabel('')
ax.set_ylabel('')
_ = ax.set_xticklabels(['Zginęli', 'Przeżyli'])

Tym razem niestety pudło - wiek nie wpływa znacząco na szanse przeżycia. Ale warto tu wspomnieć trochę o boxplotach:
    
![](box_plot.png)

Jest to bardzo wygodna i czytelna prezentacja rozkładów, trzeba się tylko do niej przyzwyczaić.

In [ ]:
def labelizer(x):
    los = {'0': 'zginęli', '1': 'przeżyli'}
    return  x[0] + ', ' + los[x[1]]

ax = mosaic(df, ['Embarked', 'Survived'], labelizer=labelizer,
            title='Zależność pomiędzy miejscem zaokrętowania, a szansami przeżycia',
            axes_label=False)
ax[0].set_size_inches((17,5))

Pasażerowie byli okrętowani w trzech różnych miejscowościach: Queenstown, Southampton i Cherbourg.
Wydaje się, że nie powinno mieć to wpływu na szanse przeżycia, ale widać, że rozkłady tej zmiennej nie są identyczne
dla poszczególnych miast. Oczywiście, mogą istnieć różne korelacje, np. pasażerowie z Cherbourg mogli częściej
wybierać pierwszą klasę, mogło być więcej kobiet wśród nich. Nie zmienia to jednak faktu, że zmienna ta będzie
przydatna.

Istnieje także taka możliwość, że pasażerowie wsiadający w Cherbourg dostali koje w tylnych częściach stref
sypialnych.

W wielu problemach uczenia maszynowego cechy, które początkowo nie wydają się zbyt ciekawe odgrywają kluczową rolę.

In [ ]:
ax = df.boxplot(['Age'], by='Pclass')
ax.set_title('Zależność wieku od klasy')
ax.set_xlabel('')
ax.set_ylabel('')
_ = ax.set_xticklabels(['Pierwsza', 'Druga', 'Trzecia'])

Jak widać, starsi pasażerowie częściej wybierali pierwszą klasę. Podobnie jak dzisiaj, ludzie stają się bogatsi z
wiekiem i ta korelacja jest logiczna.

Czasami warto także sprawdzać takie powiązania. W przypadku bezsensownych wyników można zaoszczędzić sporo czasu, jeśli wykryje się spore zabrudzenie danych / błędy w danych.

In [ ]:
df['Name']

Po spojrzeniu na imiona i nazwiska (ktoś tu chyba nie słyszał o pierwszej postaci normalnej :P), zauważamy, że każda
osoba posiada też tytuł. Cecha ta może nam się przydać.

In [ ]:
import re

df['Title'] = df['Name'].apply(lambda x: re.findall(',.*\.', x)[0][2:-1])

Wyciągamy tytuł z przed nazwiska.

In [ ]:
df['Title'].unique()

Widać, że niektóre wartości są błędne, np. Pani Elizabeth Martin miała inną kolejność tytułu/nazwiska/imienia.
Błędów jest jednak niewiele i nie będziemy się nimi przejmować. Odrzucimy każdą wartość pojawiającą się tylko raz:

In [ ]:
df2 = df.groupby('Title').filter(lambda x: len(x) > 1)

In [ ]:
ax = df2.boxplot(['Age'], by='Title')
ax.set_title('Zależność wieku od tytułu')
ax.set_xlabel('')
ax.set_ylabel('')
ax.figure.set_size_inches((20,20))

No i trafiliśmy ciekawą zależność. Zobaczymy, czy przyda się przy tworzeniu modelu.

To, co przed chwilą zrobiliśmy nazywamy _feature engineering_ (inżyneria cech? Nie znam polskiego odpowiednika).
Częściej spotkałem się z podejściem, w którym dokonuje się tego dopiero po stworzeniu pierwszych modeli, ale nic
nie szkodzi, aby trochę pobawić się wcześniej.

Jakie jeszcze cechy możemy wymyślić?

In [ ]:
def fx(sex, age):
    return sex == 'female' or age < 15

df['Boat.dibs'] = np.vectorize(fx)(df['Sex'], df['Age'])

Pierwszenstwo wstępu na szalupy ratunkowe miały kobiety i dzieci. O ile wiemy dokładnie, którzy z pasażerów to kobiety,
to nie mamy pojęcia jak zdefiniować dziecko.

Tutaj definiuję dziecko jako osobę młodszą niż 15 lat.

Po stworzeniu modeli warto tu wrócić i spróbować innej granicy lub innej metody.

In [ ]:
def fx(cabin):
    if cabin == cabin:
        return cabin[0]
    else:
        return 'NaN'

df['Deck'] = np.vectorize(fx)(df['Cabin'])

W uczeniu maszynowym warto także korzystać z zewnętrznej wiedzy (czyli zwykle Google'a).

Tutaj zaczerpniety z internetów przekrój Tytanika:

![](decks.png)

Litery występujące na bokach prawdopodobnie występują w nazwie kabiny. Dzięki temu uzyskujemy kolejną cechę
- piętro kabiny pasażera

In [ ]:
def fx(cabin):
    try:     
        return int(cabin[-1]) % 2
    except:
        return 'NaN'

df['Side'] = np.vectorize(fx)(df['Cabin'])

Dodatkowo, parzystość numeru oznaczała jego położenie na sterburcie lub bakburcie!

Niestey, jak widzieliśmy wcześniej cecha numeru koi występuje rzadko w naszym zbiorze danych. Co więcej nie wiemy,
co ona oznacza. Czy jest to koja, która została wykupiona (czyli ta widniejąca na bilecie), czy koja w której
rzeczywiście zostali zaalokowani pasażerowie.

Za tydzień zbudujemy modele i zobaczymy jak dobrze jesteśmy w stanie ocenić szanse przeżycia pasażerów.

Praca domowa.

Co ciekawego jeszce można wyciągnąć z tych danych? Sugestie:
    
- Sprawdzić jak wpływa podróżowanie z rodziną na szanse przeżycia
- Zastanowić się, czy brakujące dane można naprawić. Tu przychodzi nam z pomocą statystyka,
  należy dobrać taki rozkład, który pasuje do istniejących danych i zaaplikować go 

- Sprawdzić jak wpływa rozmiar rodziny z którą się podróżuje na szanse przeżycia

lub

- Korzystając z wiedzy zdobytej tutaj zbudować model.